In [1]:
import copy

In [2]:
def get_sml_func(r):
    rule, target = r.split(':')
    k, v = rule.split('<')
    def sml(data, wdict):
        if data[k] < int(v):
            return wdict[target](data, wdict)
        return False
    return sml
    
def get_grt_func(r):
    rule, target = r.split(':')
    k, v = rule.split('>')
    def grt(data, wdict):
        if data[k] > int(v):
            return wdict[target](data, wdict)
        return False

    return grt

def get_goto_func(r):
    def newrule(data, wdict):
        res = wdict[r](data, wdict)
        return res

    return newrule

def create_workflow(desc):
    rules = []
    for r in desc.split(','):
        if '<' in r:
            rules.append(get_sml_func(r))
        elif '>' in r:
            rules.append(get_grt_func(r))
        else:
            rules.append(get_goto_func(r))


    def f(data, wdict):
        for r in rules:
            res = r(data, wdict)
            if res:
                return res  # Rule accepted
    return f

In [3]:
def get_accept():
    def accept(data, wdict):
        wdict['accepted'].append(data)
        return True
    return accept    

def get_reject():
    def reject(data, wdict):
        wdict['rejected'].append(data)
        return True
    return reject

def generate_workflows(instructions):
    wdict = {}
    for instr in instructions:
        name, desc = instr.split('{')
        wdict[name] = create_workflow(desc[:-1])

    wdict['A'] = get_accept()
    wdict['R'] = get_reject()

    wdict['accepted'] = []
    wdict['rejected'] = []

    return wdict

In [4]:
def sum_ratings(data):
    s = 0
    for d in data:
        s += sum([v for v in d.values()])
    return s

In [5]:
def read_input(fname):
    instructions = []
    data = []
    with open(fname, 'r') as inf:
        line = inf.readline()
        while line.strip() != '':
            instructions.append(line.strip())
            line = inf.readline()

        for line in inf.readlines():
            if line.strip() == '':
                continue
            data.append({k:int(v) for k, v in [d.split('=') for d in line[1:-2].strip().split(',')]})

    return instructions, data

In [6]:
def get_sml_func_range(r):
    rule, target = r.split(':')
    k, v = rule.split('<')
    def sml(data, wdict):
        splt1 = copy.copy(data)
        splt2 = copy.copy(data)
        splt1[k] = (data[k][0], int(v)-1)
        splt2[k] = (int(v), data[k][1])
        wdict[target](splt1, wdict)
        return splt2
    return sml
    
def get_grt_func_range(r):
    rule, target = r.split(':')
    k, v = rule.split('>')
    def grt(data, wdict):
        splt1 = copy.copy(data)
        splt2 = copy.copy(data)
        splt1[k] = (int(v)+1, data[k][1])
        splt2[k] = (data[k][0], int(v))
        wdict[target](splt1, wdict)
        return splt2

    return grt

def get_goto_func_range(r):
    def newrule(data, wdict):
        wdict[r](data, wdict)
        return None

    return newrule

def create_workflow_range(desc):
    rules = []
    for r in desc.split(','):
        if '<' in r:
            rules.append(get_sml_func_range(r))
        elif '>' in r:
            rules.append(get_grt_func_range(r))
        else:
            rules.append(get_goto_func_range(r))


    def f_range(data, wdict):
        for r in rules:
            if data is None:
                raise Exception
            data = r(data, wdict)
        return None
    return f_range

In [7]:
def generate_workflows_range(instructions):
    wdict = {}
    for instr in instructions:
        name, desc = instr.split('{')
        wdict[name] = create_workflow_range(desc[:-1])

    wdict['A'] = get_accept()
    wdict['R'] = get_reject()

    wdict['accepted'] = []
    wdict['rejected'] = []

    return wdict

In [8]:
def calc_combinations(wdict):
    data = {k:(1,4000) for k in ('x', 'm', 'a', 's')}

    wdict['in'](data, wdict)

    a = 0
    r = 0
    for d in wdict['accepted']:
        val = 1
        for v in d.values():
            val *= v[1] - v[0] + 1
        a += val

    for d in wdict['rejected']:
        val = 1
        for v in d.values():
            val *= v[1] - v[0] + 1
        r += val

    # print('a', a)
    # print('r', r)

    return a

In [9]:
print('*****\nPuzzle1\n*****\n')

print('Test case\n')

instr, data = read_input('input19a.txt')
wdict = generate_workflows(instr)

for d in data:
    wdict['in'](d, wdict)

s = sum_ratings(wdict["accepted"])
print(f'Sum of rating is {s}')

assert s == 19114

print('\nPuzzle case\n')

instr, data = read_input('input19.txt')
wdict = generate_workflows(instr)

for d in data:
    wdict['in'](d, wdict)

s = sum_ratings(wdict["accepted"])
print(f'Sum of rating is {s}')

assert s == 350678

print('\n*****\nPuzzle2\n*****\n')

instr, data = read_input('input19a.txt')
wdict = generate_workflows_range(instr)

a = calc_combinations(wdict)

print(f'Number accepted combinations is {a}')

assert a == 167409079868000

print('\nPuzzle case\n')

instr, data = read_input('input19.txt')
wdict = generate_workflows_range(instr)

a = calc_combinations(wdict)

print(f'Number accepted combinations is {a}')

assert a == 124831893423809

*****
Puzzle1
*****

Test case

Sum of rating is 19114

Puzzle case

Sum of rating is 350678

*****
Puzzle2
*****

Number accepted combinations is 167409079868000

Puzzle case

Number accepted combinations is 124831893423809
